# About
This notebook contains a first simple ML model

In [50]:
# Necessary to import custom modules
import os
os.chdir("/home/jovyan/work")

from neomodel import db
import pandas as pd

from src.utils import init_connection

In [51]:
init_connection()

In [52]:
cypher = """
MATCH 
    (t:Team)-[:SCORED]->(s:Score)-[:IN_GAME]->(g:Game)<-[:IN_GAME]-(s2:Score)<-[:SCORED]-(t2:Team), (sea:Season), (arena_game:Arena), (arena_t:Arena)
WHERE 
    (g)-[:TOOK_PLACE_IN]->(sea)
AND
	(g)-[:LOCATED_IN]->(arena_game)
AND
	(t)-[:HAVE_HOME_COURT_AT]->(arena_t)
RETURN 
    t.name as team, 
    sum(
    	CASE WHEN s.score > s2.score 
        AND g.game_type = "regular_season" 
        AND arena_game.name = arena_t.name
        THEN 1 ELSE 0 END) 
    as wins_reg_home,
    sum(
    	CASE WHEN s.score > s2.score 
        AND g.game_type = "regular_season" 
        AND arena_game.name <> arena_t.name
        THEN 1 ELSE 0 END) 
    as wins_reg_away,
    sum(
    	CASE WHEN s.score < s2.score 
        AND g.game_type = "regular_season" 
        AND arena_game.name = arena_t.name
        THEN 1 ELSE 0 END) 
    as losses_reg_home,
    sum(
    	CASE WHEN s.score < s2.score
        AND g.game_type = "regular_season" 
        AND arena_game.name <> arena_t.name
        THEN 1 ELSE 0 END) as losses_reg_away,
	sea.name as season
ORDER BY 
    team, season
"""

In [53]:
data, columns = db.cypher_query(cypher)

In [54]:
df_results = pd.DataFrame(data=data, columns=columns)

In [55]:
df_test = df_results.pivot(index="team", columns="season")

In [56]:
cols = []
for a, b in zip(df_test.columns.get_level_values(0), df_test.columns.get_level_values(1)):
    cols.append(a+"_"+b)

In [57]:
cols

['wins_reg_home_2015/2016',
 'wins_reg_home_2016/2017',
 'wins_reg_home_2017/2018',
 'wins_reg_away_2015/2016',
 'wins_reg_away_2016/2017',
 'wins_reg_away_2017/2018',
 'losses_reg_home_2015/2016',
 'losses_reg_home_2016/2017',
 'losses_reg_home_2017/2018',
 'losses_reg_away_2015/2016',
 'losses_reg_away_2016/2017',
 'losses_reg_away_2017/2018']

In [60]:
df_test.columns = cols

In [61]:
df_test.head()

,wins_reg_home_2015/2016,wins_reg_home_2016/2017,wins_reg_home_2017/2018,wins_reg_away_2015/2016,wins_reg_away_2016/2017,wins_reg_away_2017/2018,losses_reg_home_2015/2016,losses_reg_home_2016/2017,losses_reg_home_2017/2018,losses_reg_away_2015/2016,losses_reg_away_2016/2017,losses_reg_away_2017/2018
team,,,,,,,,,,,,
Atlanta Hawks,30,25,16,22,20,8,16,19,25,24,24,33
Boston Celtics,30,35,37,20,27,29,14,16,15,24,22,20
Brooklyn Nets,14,13,15,7,7,13,27,28,26,34,34,28
Charlotte Hornets,32,22,21,19,14,15,12,19,20,26,27,26
Chicago Bulls,26,25,17,16,18,10,15,19,24,25,26,31


#### Extract Matchup

```
MATCH 
	(t:Team)-[:SCORED]->(s:Score)-[:IN_GAME]->(g:Game)<-[:IN_GAME]-(s2:Score)<-[:SCORED]-(t2:Team), (sea:Season)
WHERE 
    (g)-[:TOOK_PLACE_IN]->(sea)
AND
	sea.name = "2017/2018"
AND
	g.game_type = "regular_season"
WITH g, collect(t.name) as teams, collect(s.score) as scores
RETURN
	HEAD(teams) as teamA, HEAD(TAIL(scores)) as scoreA, HEAD(TAIL(teams)) as teamB, HEAD(scores) as scoreB
```

In [62]:
cypher = """
MATCH 
	(t:Team)-[:SCORED]->(s:Score)-[:IN_GAME]->(g:Game)<-[:IN_GAME]-(s2:Score)<-[:SCORED]-(t2:Team), (sea:Season)
WHERE 
    (g)-[:TOOK_PLACE_IN]->(sea)
AND
	sea.name = "2017/2018"
AND
	g.game_type = "regular_season"
WITH g, collect(t.name) as teams, collect(s.score) as scores
RETURN
	HEAD(teams) as teamA, HEAD(TAIL(scores)) as scoreA, HEAD(TAIL(teams)) as teamB, HEAD(scores) as scoreB
"""

In [63]:
data, cols = db.cypher_query(cypher)

In [64]:
df_matches = pd.DataFrame(data=data, columns=cols)

In [65]:
df_matches["teamAWinner"] =df_matches["scoreA"] > df_matches["scoreB"]

In [66]:
df_test.head().reset_index()

,team,wins_reg_home_2015/2016,wins_reg_home_2016/2017,wins_reg_home_2017/2018,wins_reg_away_2015/2016,wins_reg_away_2016/2017,wins_reg_away_2017/2018,losses_reg_home_2015/2016,losses_reg_home_2016/2017,losses_reg_home_2017/2018,losses_reg_away_2015/2016,losses_reg_away_2016/2017,losses_reg_away_2017/2018
0,Atlanta Hawks,30,25,16,22,20,8,16,19,25,24,24,33
1,Boston Celtics,30,35,37,20,27,29,14,16,15,24,22,20
2,Brooklyn Nets,14,13,15,7,7,13,27,28,26,34,34,28
3,Charlotte Hornets,32,22,21,19,14,15,12,19,20,26,27,26
4,Chicago Bulls,26,25,17,16,18,10,15,19,24,25,26,31


In [67]:
df_matches.head()

,teamA,scoreA,teamB,scoreB,teamAWinner
0,Los Angeles Clippers,121,Golden State Warriors,105,True
1,Memphis Grizzlies,88,Sacramento Kings,106,False
2,New York Knicks,86,Miami Heat,115,False
3,Utah Jazz,103,Oklahoma City Thunder,89,True
4,Los Angeles Clippers,107,Atlanta Hawks,108,False


In [68]:
pd.merge(df_matches, df_test.reset_index(), left_on="teamA", right_on="team").head()

,teamA,scoreA,teamB,scoreB,teamAWinner,team,wins_reg_home_2015/2016,wins_reg_home_2016/2017,wins_reg_home_2017/2018,wins_reg_away_2015/2016,wins_reg_away_2016/2017,wins_reg_away_2017/2018,losses_reg_home_2015/2016,losses_reg_home_2016/2017,losses_reg_home_2017/2018,losses_reg_away_2015/2016,losses_reg_away_2016/2017,losses_reg_away_2017/2018
0,Los Angeles Clippers,121,Golden State Warriors,105,True,Los Angeles Clippers,33,31,24,22,23,18,13,16,19,20,19,21
1,Los Angeles Clippers,107,Atlanta Hawks,108,False,Los Angeles Clippers,33,31,24,22,23,18,13,16,19,20,19,21
2,Los Angeles Clippers,105,New York Knicks,128,False,Los Angeles Clippers,33,31,24,22,23,18,13,16,19,20,19,21
3,Los Angeles Clippers,104,Denver Nuggets,109,False,Los Angeles Clippers,33,31,24,22,23,18,13,16,19,20,19,21
4,Los Angeles Clippers,104,Portland Trail Blazers,96,True,Los Angeles Clippers,33,31,24,22,23,18,13,16,19,20,19,21


In [69]:
df_ml = pd.merge(
    pd.merge(df_matches, df_test.reset_index(), left_on="teamA", right_on="team"),
    df_test.reset_index(), 
    left_on="teamB", 
    right_on="team",
    suffixes=["_A", "_B"]
).drop(
    ["scoreA", "scoreB", "teamA", "teamB", "team_A", "team_B", "wins_reg_home_2017/2018_A", "wins_reg_away_2017/2018_A", "wins_reg_home_2017/2018_B", "wins_reg_away_2017/2018_B", "losses_reg_home_2017/2018_A", "losses_reg_away_2017/2018_A", "losses_reg_home_2017/2018_B", "losses_reg_away_2017/2018_B"], 
    axis=1)

In [70]:
df_ml.head()

,teamAWinner,wins_reg_home_2015/2016_A,wins_reg_home_2016/2017_A,wins_reg_away_2015/2016_A,wins_reg_away_2016/2017_A,losses_reg_home_2015/2016_A,losses_reg_home_2016/2017_A,losses_reg_away_2015/2016_A,losses_reg_away_2016/2017_A,wins_reg_home_2015/2016_B,wins_reg_home_2016/2017_B,wins_reg_away_2015/2016_B,wins_reg_away_2016/2017_B,losses_reg_home_2015/2016_B,losses_reg_home_2016/2017_B,losses_reg_away_2015/2016_B,losses_reg_away_2016/2017_B
0,True,33,31,22,23,13,16,20,19,50,45,38,38,5,5,13,11
1,True,33,31,22,23,13,16,20,19,50,45,38,38,5,5,13,11
2,False,26,26,16,19,17,18,27,25,50,45,38,38,5,5,13,11
3,True,18,19,14,12,23,22,27,29,50,45,38,38,5,5,13,11
4,False,24,30,16,25,17,16,25,22,50,45,38,38,5,5,13,11


In [71]:
from sklearn.linear_model import LogisticRegression

In [72]:
lr = LogisticRegression()

In [73]:
from sklearn.model_selection import train_test_split

In [74]:
X_train, X_test, y_train, y_test = train_test_split(
    df_ml.drop("teamAWinner", axis=1), 
    df_ml["teamAWinner"]
)

lr.fit(X_train, y_train)

lr.score(X_test, y_test)

0.68597560975609762

# Average Win Margin per Team 

## Validierung der eigentlichen Cypher-Query

In [75]:
cypher = """
MATCH 
	(t:Team)-[:SCORED]->(s:Score)-[:IN_GAME]->(g:Game)<-[:IN_GAME]-(s2:Score)<-[:SCORED]-(t2:Team), (sea:Season)
WHERE 
    (g)-[:TOOK_PLACE_IN]->(sea)
AND
	sea.name = "2017/2018"
AND
	g.game_type = "regular_season"
WITH g, collect(t.name) as teams, collect(s.score) as scores
RETURN
	HEAD(teams) as teamA, HEAD(TAIL(scores)) as scoreA, HEAD(TAIL(teams)) as teamB, HEAD(scores) as scoreB
"""

In [76]:
data, cols = db.cypher_query(cypher)

In [77]:
df_matchesX = pd.DataFrame(data=data, columns=cols)

In [78]:
df_matchesX.head()

,teamA,scoreA,teamB,scoreB
0,Los Angeles Clippers,121,Golden State Warriors,105
1,Memphis Grizzlies,88,Sacramento Kings,106
2,New York Knicks,86,Miami Heat,115
3,Utah Jazz,103,Oklahoma City Thunder,89
4,Los Angeles Clippers,107,Atlanta Hawks,108


In [79]:
df_matches2 = df_matchesX

In [80]:
df_matches2["scoreMargin"] = abs(df_matches2["scoreA"]-df_matches2["scoreB"])

In [81]:
df_matches2.head()

,teamA,scoreA,teamB,scoreB,scoreMargin
0,Los Angeles Clippers,121,Golden State Warriors,105,16
1,Memphis Grizzlies,88,Sacramento Kings,106,18
2,New York Knicks,86,Miami Heat,115,29
3,Utah Jazz,103,Oklahoma City Thunder,89,14
4,Los Angeles Clippers,107,Atlanta Hawks,108,1


In [82]:
a = ["Atlanta Hawks"]

In [83]:
df_onlyBOSmatches = df_matches2[(df_matches2['teamA'].isin(a)) | (df_matches2['teamB'].isin(a)) ]

In [84]:
df_onlyBOSmatches["scoreMargin"].mean()

10.865853658536585

## Actual Cypher Query

In [203]:
cypher = """
MATCH 
    (t:Team)-[:SCORED]->(s:Score)-[:IN_GAME]->(g:Game)<-[:IN_GAME]-(s2:Score)<-[:SCORED]-(t2:Team), (sea:Season)
WHERE 
    (g)-[:TOOK_PLACE_IN]->(sea)
AND
	g.game_type= "regular_season"
RETURN 
    t.name as team, 
    avg(abs(s.score - s2.score)) as scoreMargin,
    sea.name as season
ORDER BY 
    team, season

"""

In [204]:
data, cols = db.cypher_query(cypher)

In [205]:
df_scoreMargin = pd.DataFrame(data=data, columns=cols)

In [206]:
df_scoreMargin.head()

,team,scoreMargin,season
0,Atlanta Hawks,11.717391,2015/2016
1,Atlanta Hawks,11.125000,2016/2017
2,Atlanta Hawks,10.865854,2017/2018
3,Boston Celtics,10.806818,2015/2016
4,Boston Celtics,10.370000,2016/2017


In [207]:
df_scoreMargin_test = df_scoreMargin.pivot(index="team", columns="season")

In [208]:
df_scoreMargin_test.head()

scoreMargin                      
season              2015/2016  2016/2017  2017/2018
team                                               
Atlanta Hawks       11.717391  11.125000  10.865854
Boston Celtics      10.806818  10.370000   9.693069
Brooklyn Nets       10.743902  12.195122  10.231707
Charlotte Hornets   11.449438  10.195122  11.768293
Chicago Bulls        9.865854  12.079545  12.219512

In [209]:
cols = []
for a, b in zip(df_scoreMargin_test.columns.get_level_values(0), df_scoreMargin_test.columns.get_level_values(1)):
    cols.append(a+"_"+b)

In [210]:
cols

['scoreMargin_2015/2016', 'scoreMargin_2016/2017', 'scoreMargin_2017/2018']

In [211]:
df_scoreMargin_test.columns = cols

In [212]:
df_scoreMargin_test.head()

,scoreMargin_2015/2016,scoreMargin_2016/2017,scoreMargin_2017/2018
team,,,
Atlanta Hawks,11.717391,11.125000,10.865854
Boston Celtics,10.806818,10.370000,9.693069
Brooklyn Nets,10.743902,12.195122,10.231707
Charlotte Hornets,11.449438,10.195122,11.768293
Chicago Bulls,9.865854,12.079545,12.219512


In [213]:
df_ml_1 = pd.merge(
    pd.merge(df_matches, df_test.reset_index(), left_on="teamA", right_on="team"),
    df_test.reset_index(), 
    left_on="teamB", 
    right_on="team",
    suffixes=["_A", "_B"]
)

In [214]:
df_ml_2 = pd.merge(
    pd.merge(df_ml_1, df_scoreMargin_test.reset_index(), left_on="teamA", right_on="team"),
    df_scoreMargin_test.reset_index(), 
    left_on="teamB", 
    right_on="team",
    suffixes=["_A", "_B"]
).drop(["scoreA", "scoreB", "teamA", "teamB", "team_A", "team_B", "wins_reg_home_2017/2018_A", "wins_reg_away_2017/2018_A", "losses_reg_home_2017/2018_A", "losses_reg_home_2017/2018_A",  "wins_reg_home_2017/2018_B", "wins_reg_away_2017/2018_B", "losses_reg_home_2017/2018_B", "losses_reg_home_2017/2018_B", "scoreMargin_2017/2018_A", "scoreMargin_2017/2018_B"], axis=1)

## Score Margin splitted into home and away

In [221]:
cypher = """
MATCH 
    (t:Team)-[:SCORED]->(s:Score)-[:IN_GAME]->(g:Game)<-[:IN_GAME]-(s2:Score)<-[:SCORED]-(t2:Team), (sea:Season)
WHERE 
    (g)-[:TOOK_PLACE_IN]->(sea)
AND
	g.game_type= "regular_season"
RETURN 
    t.name as team, 
    sea.name as season,
    CASE WHEN s.score > s2.score THEN avg(abs(s.score - s2.score)) ELSE 0 END as scoreMarginWins,
    CASE WHEN s.score < s2.score THEN avg(abs(s.score - s2.score)) ELSE 0 END as scoreMarginLosses

ORDER BY 
    team, season

"""

In [222]:
data, cols = db.cypher_query(cypher)

In [223]:
df_scoreMargin = pd.DataFrame(data=data, columns=cols)

In [224]:
df_scoreMargin.head()

,team,season,scoreMarginWins,scoreMarginLosses
0,Atlanta Hawks,2015/2016,13.115385,0.00000
1,Atlanta Hawks,2015/2016,0.000000,9.90000
2,Atlanta Hawks,2016/2017,10.022222,0.00000
3,Atlanta Hawks,2016/2017,0.000000,12.27907
4,Atlanta Hawks,2017/2018,9.250000,0.00000


In [225]:
df_scoreMargin_home_away = df_scoreMargin.groupby(["team", "season"]).sum().reset_index()

In [226]:
df_scoreMargin_test = df_scoreMargin_home_away.pivot(index="team", columns="season")

In [227]:
cols = []
for a, b in zip(df_scoreMargin_test.columns.get_level_values(0), df_scoreMargin_test.columns.get_level_values(1)):
    cols.append(a+"_"+b)

In [228]:
df_scoreMargin_test.columns = cols

## Validation of numbers

In [229]:
df_scoreMargin_home_away[df_scoreMargin_home_away["team"] == "Houston Rockets"]

,team,season,scoreMarginWins,scoreMarginLosses
30,Houston Rockets,2015/2016,10.047619,11.111111
31,Houston Rockets,2016/2017,13.688525,10.906250
32,Houston Rockets,2017/2018,13.118421,11.782609


(4+5+1+18+7+11+2+3+17+13)

(6+10+6+10+16+8+8)

In [230]:
test = [14,5,1,18,7,11,2,3,17,13,6,10,6,10,16,8,8]

In [231]:
import numpy as np
np.mean(test)

9.117647058823529

# Models

##  Logistic Regression

In [285]:
from sklearn.linear_model import LogisticRegression

In [286]:
lr = LogisticRegression()

In [287]:
from sklearn.model_selection import train_test_split

In [290]:
X_train, X_test, y_train, y_test = train_test_split(
    df_ml_2.drop("teamAWinner", axis=1), 
    df_ml_2["teamAWinner"]
)

lr.fit(X_train, y_train)

lr.score(X_test, y_test)

0.67073170731707321

In [291]:
cross_val_score(logreg, X_test, y_test, cv=10, scoring="roc_auc").mean()

0.68559065934065933

## Linear SVC

In [302]:
ls = LinearSVC()

In [310]:
ls.fit(X_train, y_train)

ls.score(X_test, y_test)

0.41339491916859122

## MLP

In [219]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

In [292]:
X_train, X_test, y_train, y_test = train_test_split(
    df_ml_2.drop("teamAWinner", axis=1), 
    df_ml_2["teamAWinner"]
)
clf.fit(X_train, y_train) 
clf.score(X_test, y_test)

0.66463414634146345

In [293]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [294]:
# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(
    df_ml_2.drop("teamAWinner", axis=1), 
    df_ml_2["teamAWinner"], 
    test_size=test_size,
    random_state=seed
)


In [295]:
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [296]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [297]:
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 65.59%


In [298]:
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, y_pred)
print(confusion)

[[196  70]
 [ 79  88]]
